<a href="https://colab.research.google.com/github/chetannitk/BeerDataScienceAssignment/blob/master/notebooks/04-k-mean-clustering-to-find-similar-beer-drinkers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-success">
    <h1> Beer Data Science Assignment <br/></h1>
    K-mean clustering to find similar group of beer drinkers.
</div>


In [1]:
## Java version should be 1.8
!java -version

openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


In [2]:
!sudo apt install openjdk-8-jdk


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 10 newly installed, 0 to remove and 31 not upgraded.
Need to get 40.7 MB of archives.
After this operation, 153 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [1

In [3]:
!sudo update-alternatives --config java
## Select java 8 option 

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [1]:
from IPython.display import HTML
import pandas as pd
import os
import numpy as np
from datetime import datetime
import wordcloud
import warnings
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
warnings.filterwarnings('ignore')

#BASE_PATH = "../"
#INPUT_DATA_PATH = os.path.join(BASE_PATH, "data/raw/BeerDataScienceProject.csv")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  # Remove the CWD from sys.path while we load stuff.


In [2]:
## Upload BeerDataScienceProject.csv file from local system.
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/data/BeerDataScienceProject.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!ls -lrt /tmp

total 405188
drwx------ 2 root root      4096 Jun  1 11:56 tmpbh64ylhv
srw------- 1 root root         0 Jun  1 11:56 drivefs_ipc.0
srw------- 1 root root         0 Jun  1 11:56 drivefs_ipc.0_shell
drwxr-xr-x 3 root root      4096 Jun  1 11:57 tfhub_modules
drwxr-xr-x 2 root root      4096 Jun  1 11:58 hsperfdata_root
drwxr-xr-x 2 root root      4096 Jun  1 11:58 spark-8e687d17-be98-4cbe-82b5-8811b6623ca0
drwxr-xr-x 2 root root      4096 Jun  1 11:58 blockmgr-2a1f6359-c905-4c38-b11b-9ab0a5da8a4f
drwx------ 4 root root      4096 Jun  1 11:58 spark-c7078239-b26b-4e2a-8a06-7872077760ef
-rw-r--r-- 1 root root 414885425 Jun  1 14:14 BeerDataScienceProject.csv


In [0]:
import io
df_beer_raw = pd.read_csv('/tmp/BeerDataScienceProject.csv', encoding="ISO-8859-1")

In [5]:
df_beer_raw.shape

(528870, 13)

In [6]:
df_beer_raw.dtypes

beer_ABV              float64
beer_beerId             int64
beer_brewerId           int64
beer_name              object
beer_style             object
review_appearance     float64
review_palette        float64
review_overall        float64
review_taste          float64
review_profileName     object
review_aroma          float64
review_text            object
review_time             int64
dtype: object

### Data Cleaning
```In the below cell we can see that there are some columns where data is missing. 
   We need to remove those data points from our data set.
```

In [7]:
df_beer_raw.isna().sum()

beer_ABV              20280
beer_beerId               0
beer_brewerId             0
beer_name                 0
beer_style                0
review_appearance         0
review_palette            0
review_overall            0
review_taste              0
review_profileName      115
review_aroma              0
review_text             119
review_time               0
dtype: int64

In [0]:
df_beer = df_beer_raw.dropna()

In [9]:
df_beer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508358 entries, 0 to 528796
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   beer_ABV            508358 non-null  float64
 1   beer_beerId         508358 non-null  int64  
 2   beer_brewerId       508358 non-null  int64  
 3   beer_name           508358 non-null  object 
 4   beer_style          508358 non-null  object 
 5   review_appearance   508358 non-null  float64
 6   review_palette      508358 non-null  float64
 7   review_overall      508358 non-null  float64
 8   review_taste        508358 non-null  float64
 9   review_profileName  508358 non-null  object 
 10  review_aroma        508358 non-null  float64
 11  review_text         508358 non-null  object 
 12  review_time         508358 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 54.3+ MB


In [0]:
df_beer = df_beer.reset_index()
df_beer['id'] = df_beer.index

## Data Changes in Raw and Clean dataset

In [11]:
print("Raw data size: ", df_beer_raw.shape[0])
print("Clean data size: ", df_beer.shape[0])
print("Data points difference: ", df_beer_raw.shape[0]-df_beer.shape[0])

Raw data size:  528870
Clean data size:  508358
Data points difference:  20512


### Describe the categorical features

In [12]:
df_beer.describe(include=[np.object])

,beer_name,beer_style,review_profileName,review_text
count,508358,508358,508358,508358
unique,14028,104,22563,508007
top,Sierra Nevada Celebration Ale,American IPA,northyorksammy,#NAME?
freq,2998,42092,1732,90


### Describe the numerical features

In [13]:
df_beer.describe(include=[np.number])

,index,beer_ABV,beer_beerId,beer_brewerId,review_appearance,review_palette,review_overall,review_taste,review_aroma,review_time,id
count,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,5.083580e+05,508358.000000
mean,263669.490377,7.017400,21824.120840,2534.273207,3.872676,3.768993,3.840805,3.775327,3.827646,1.226175e+09,254178.500000
std,152590.091973,2.204528,22124.969107,5237.843825,0.601764,0.682355,0.706408,0.665586,0.715121,7.530843e+07,146750.458409
min,0.000000,0.010000,5.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,8.843904e+08,0.000000
25%,131284.250000,5.300000,1673.000000,132.000000,3.500000,3.500000,3.500000,3.500000,3.500000,1.177202e+09,127089.250000
50%,263756.500000,6.500000,13850.000000,392.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.241503e+09,254178.500000
75%,395689.750000,8.500000,40418.000000,1315.000000,4.000000,4.000000,4.500000,4.000000,4.500000,1.289073e+09,381267.750000
max,528796.000000,57.700000,77310.000000,27980.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.326277e+09,508357.000000


In [0]:
NUMERICAL_ATTRIBUTE = ['beer_ABV', 'review_appearance', 'review_palette', 
                       'review_overall', 'review_taste', 'review_aroma', 'review_time']
CATEGORICAL_ATTRIBUTE = ['beer_name', 'beer_style', 'review_profileName', 'review_text']

In [15]:
df_beer.sample(4)

,index,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time,id
226705,234885,10.2,6512,1628,Imperial Russian Stout,Russian Imperial Stout,4.0,4.0,4.5,4.5,Billolick,5.0,On tap at the venerable Blind Tiger ale house ...,1110425548,226705
392279,407101,6.0,6322,694,Tröegs Hopback Amber Ale,American Amber / Red Ale,4.0,4.0,4.0,4.0,9InchNails,4.0,Creamy white head. Cloudy amber color. Looks r...,1318751035,392279
248696,257372,8.0,5253,841,Vapeur En Folie,Belgian Strong Pale Ale,4.0,2.0,3.0,4.0,NeroFiddled,2.5,750 ml bottle. Pours a slightly hazy gold body...,1056863320,248696
452479,470533,7.0,16637,392,Black Hole,Old Ale,4.0,4.0,4.0,4.0,edchicma,4.0,Another good offering from Weyerbacher. Black ...,1145021945,452479


<div class="alert alert-info">
Convert the review timestamp to datetime format
</div>

In [0]:
df_beer['d_review_time'] = df_beer.review_time.apply(lambda t: datetime.fromtimestamp(t))

In [0]:
df_beer['d_review_time_year'] = df_beer['d_review_time'].dt.year
df_beer['d_review_time_month'] = df_beer['d_review_time'].dt.month

<div class="alert alert-success">
Q7) How do find similar beer drinkers by using written reviews only?   
</div>

In [18]:
!pip install tensorflow_text

In [19]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
tf.executing_eagerly()

True

In [0]:
embed = hub.load("https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2")

In [21]:
!pip install pyspark

In [22]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
import pyspark
from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import *


%matplotlib inline

print (pyspark.__version__)

2.4.5


In [0]:
sc = SparkContext("local[*]", "First App1")
spark = SparkSession(sc)

In [0]:
train = df_beer.review_text.tolist()
list_train = [train[i:i+100] for i in range(0,len(train),100)]

In [25]:
from tqdm import tqdm
data = []
for d in tqdm(list_train):
    data.extend(embed(d).numpy())

100%|██████████| 5084/5084 [00:30<00:00, 166.12it/s]


In [0]:
df = pd.DataFrame(data, columns=["c"+str(i) for i in range(50)])
df['id'] = df.index
df = spark.createDataFrame(df) 


In [28]:
df.show()

+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

In [29]:
FEATURES_COL = ["c"+str(i) for i in range(50)]
for col in df.columns:
    if col in FEATURES_COL:
        df = df.withColumn(col,df[col].cast('float'))
df.show()

+----------+------------+------------+----------+------------+-----------+------------+------------+-----------+----------+-----------+------------+------------+-------------+-----------+-----------+------------+------------+-------------+-----------+-----------+-----------+------------+----------+-----------+------------+------------+-----------+-----------+------------+------------+------------+----------+-----------+-------------+------------+------------+------------+------------+-----------+------------+------------+-----------+------------+-----------+------------+-------------+------------+------------+------------+---+
|        c0|          c1|          c2|        c3|          c4|         c5|          c6|          c7|         c8|        c9|        c10|         c11|         c12|          c13|        c14|        c15|         c16|         c17|          c18|        c19|        c20|        c21|         c22|       c23|        c24|         c25|         c26|        c27|        c28|    

In [30]:
df = df.na.drop()
df.show()

+----------+------------+------------+----------+------------+-----------+------------+------------+-----------+----------+-----------+------------+------------+-------------+-----------+-----------+------------+------------+-------------+-----------+-----------+-----------+------------+----------+-----------+------------+------------+-----------+-----------+------------+------------+------------+----------+-----------+-------------+------------+------------+------------+------------+-----------+------------+------------+-----------+------------+-----------+------------+-------------+------------+------------+------------+---+
|        c0|          c1|          c2|        c3|          c4|         c5|          c6|          c7|         c8|        c9|        c10|         c11|         c12|          c13|        c14|        c15|         c16|         c17|          c18|        c19|        c20|        c21|         c22|       c23|        c24|         c25|         c26|        c27|        c28|    

In [31]:
##Create a features column to be used in the clustering
vecAssembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
df_kmeans = vecAssembler.transform(df).select('id', 'features')
df_kmeans.show()

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[0.40614145994186...|
|  1|[0.41943061351776...|
|  2|[0.46863827109336...|
|  3|[0.52075523138046...|
|  4|[0.53998696804046...|
|  5|[0.14735037088394...|
|  6|[0.55431103706359...|
|  7|[0.43932449817657...|
|  8|[0.26950046420097...|
|  9|[0.45464089512825...|
| 10|[0.57825732231140...|
| 11|[0.29450130462646...|
| 12|[0.30755177140235...|
| 13|[0.34669226408004...|
| 14|[0.34361752867698...|
| 15|[0.54438114166259...|
| 16|[0.38034585118293...|
| 17|[0.58402925729751...|
| 18|[0.52101546525955...|
| 19|[0.57817655801773...|
+---+--------------------+
only showing top 20 rows



## Optimize choice of k (Number of cluster center)
To optimize k we cluster a fraction of the data for different choices of k and look for an "elbow" in the cost function.

In [0]:
from tqdm import tqdm

cost = np.zeros(20)
for k in tqdm(range(2,20)):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(df_kmeans.sample(False,0.1, seed=42))
    cost[k] = model.computeCost(df_kmeans) 

  0%|          | 0/18 [00:00<?, ?it/s]

In [0]:
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,20),cost[2:20])
ax.set_xlabel('k')
ax.set_ylabel('cost')

## Train the Model

In [0]:
k = 10
kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
model = kmeans.fit(df_kmeans)
centers = model.clusterCenters()

print("Cluster Centers: ")
for center in centers:
    print(center)

## Assign cluster to rows

In [0]:
transformed = model.transform(df_kmeans).select('id', 'prediction')
rows = transformed.collect()
print(rows[:3])

In [0]:
df_pred = spark.createDataFrame(rows)
df_pred.show()

## Join the prediction with the original data


In [0]:
df_pred = df_pred.toPandas()
